In [3]:
%cd /content/drive/MyDrive/Master Thesis/01 Data Acquisition

/content/drive/MyDrive/Master Thesis/01 Data Acquisition


In [ ]:
!pip install pandas
!pip install geopandas


In [4]:
# Load packages
import os, io, json
import pandas as pd
import geopandas as gpd
from ast import literal_eval
from shapely.geometry import shape, mapping as shapely_mapping
from shapely import wkt
import folium
from PIL import Image, ImageDraw
from __future__ import print_function
from ipywidgets import interact,interact_manual,HBox,Output,Tab
import time

import cv2
import random
import re
import sys
import multiprocessing.dummy as mp
import numpy as np

In [61]:
# Create an unique id based on position of parking lot
def parking_name(row, ts):
    try:
        pt = row
        a = round(pt[0], 7)
        b = round(pt[1], 7)
    except:
        return np.nan

    return f"{a}_{b}_ts{ts}"

# Find tags containing truck or asphalt surface
def find_tags(df2):
    ls = []
    for j in range(0, len(df2)):
        index = True
        for i in df2.iloc[j,:].all_tags:
            #if i["key"] == "surface" and i["value"] == "asphalt":
            if i["key"] == "name" and i["value"].lower().__contains__("truck"):   # also other tags might be interesting
                ls.append(True)
                index = False

        if index:
            ls.append(False)

    return ls

# Get the first point of polygon list as tuple of floats
def get_coordinate(geometry):
    try:
        coordinates = re.split(r", ", geometry[9:])
        first_coordinates = (float(coordinates[1]), float(coordinates[0]))
    except:
        return np.nan

    return first_coordinates

In [6]:
# SPECIFY WHICH TILE SERVER SHOULD BE USED
# 1 = World Imagery from Esri Satellite (3-5 years old) - https://www.arcgis.com/home/item.html?id=10df2279f9684e4a9f6a7f08febac2a9
# 2 = World Imagery (Clarity) from Esri Satellite (more than 3-5 years old, however higher image quality)
TS = 2

In [60]:
geometry_example = "POLYGON((7.2291172, 51.1589647, 7.2290786, 51.1590647, ...))"
coordinates = re.split(r", ", geometry_example[9:])
coordinates = (float(coordinates[1]), float(coordinates[0]))
coordinates

(51.1589647, 7.2291172)

In [68]:
# Loading the data and pre-processing
# This part needs to be adapted according to database used

# THIS CAN TAKE UP SOME TIME DEPENDING ON DATA SIZE

start_time = time.localtime()
start_t = time.time()
print("Started at:", time.asctime(start_time))


df = pd.read_json('/content/drive/MyDrive/Master Thesis/01 Data Acquisition/truck_parking.json', orient='records')

df


Started at: Sat Jan 27 17:16:47 2024


,osm_version,osm_way_id,name,osm_timestamp,geometry,all_tags,hgv,parking,capacity,amenity
0,6,1528495,LKW-Parkplatz,2023-12-28T17:21:49Z,"POLYGON((7.2291172, 51.1589647, 7.2290786, 51....","@id=""relation/1528495"", access=""yes"", amenity=...",designated,surface,14.0,parking
1,3,3280694,None,2019-11-07T10:29:48Z,"POLYGON((12.1328729, 50.4333758, 12.1330065, 5...","@id=""relation/3280694"", amenity=""parking"", fee...",designated,surface,NaN,parking
2,3,11487370,None,2021-10-06T17:22:47Z,"POLYGON((9.5188488, 51.2689322, 9.5188025, 51....","@id=""relation/11487370"", amenity=""parking"", hg...",designated,None,NaN,parking
3,3,12605677,LKW-Parkplatz,2023-12-28T17:21:49Z,"POLYGON((7.2295367, 51.1577342, 7.2296268, 51....","@id=""relation/12605677"", access=""yes"", amenity...",designated,surface,NaN,parking
4,2,12969501,None,2022-07-13T11:35:46Z,"POLYGON((10.2129962, 48.6891887, 10.2129365, 4...","@id=""relation/12969501"", amenity=""parking"", ca...",yes,surface,32.0,parking
...,...,...,...,...,...,...,...,...,...,...
943,1,1231913595,None,2023-12-17T14:50:05Z,"POLYGON((8.5529195, 49.6451625, 8.5528658, 49....","@id=""way/1231913595"", access=""yes"", amenity=""p...",designated,surface,36.0,parking
944,1,1231913596,None,2023-12-17T14:50:05Z,"POLYGON((8.5534208, 49.6449368, 8.5535815, 49....","@id=""way/1231913596"", access=""yes"", amenity=""p...",designated,surface,36.0,parking
945,1,1239658430,None,2024-01-14T16:25:58Z,"POLYGON((13.5723058, 51.0616245, 13.5723061, 5...","@id=""way/1239658430"", access=""yes"", amenity=""p...",designated,street_side,NaN,parking
946,1,1240008646,None,2024-01-15T22:23:31Z,"POLYGON((8.720233, 51.6850829, 8.7202571, 51.6...","@id=""way/1240008646"", access=""private"", amenit...",yes,surface,NaN,parking


In [72]:
df["pt"] = df["geometry"].transform(lambda x: get_coordinate(x)) # get first point of coordinates list
df["id"] = [parking_name(x, TS) for x in df["pt"]] # create id based on coordinates

df['geomi'] = df['geometry'].transform(lambda x: convert_to_wkt(x))
#df['geometry'] = df['geometry'].apply(wkt.loads)
#df = df[["id", "pt", "geometry", "all_tags"]]
#df = df.dropna()

#temp = df
#real_polys = gpd.GeoDataFrame(temp, geometry="geometry")

#end_t = time.time()
#print("Computation time (Min.):", (end_t-start_t)/60)
df

,osm_version,osm_way_id,name,osm_timestamp,geometry,all_tags,hgv,parking,capacity,amenity,pt,id,geomi
0,6,1528495,LKW-Parkplatz,2023-12-28T17:21:49Z,"POLYGON((7.2291172, 51.1589647, 7.2290786, 51....","@id=""relation/1528495"", access=""yes"", amenity=...",designated,surface,14.0,parking,"(51.1589647, 7.2291172)",51.1589647_7.2291172_ts2,"POLYGON((7.2291172, 51.1589647, 7.2290786, 51...."
1,3,3280694,None,2019-11-07T10:29:48Z,"POLYGON((12.1328729, 50.4333758, 12.1330065, 5...","@id=""relation/3280694"", amenity=""parking"", fee...",designated,surface,NaN,parking,"(50.4333758, 12.1328729)",50.4333758_12.1328729_ts2,"POLYGON((12.1328729, 50.4333758, 12.1330065, 5..."
2,3,11487370,None,2021-10-06T17:22:47Z,"POLYGON((9.5188488, 51.2689322, 9.5188025, 51....","@id=""relation/11487370"", amenity=""parking"", hg...",designated,None,NaN,parking,"(51.2689322, 9.5188488)",51.2689322_9.5188488_ts2,"POLYGON((9.5188488, 51.2689322, 9.5188025, 51...."
3,3,12605677,LKW-Parkplatz,2023-12-28T17:21:49Z,"POLYGON((7.2295367, 51.1577342, 7.2296268, 51....","@id=""relation/12605677"", access=""yes"", amenity...",designated,surface,NaN,parking,"(51.1577342, 7.2295367)",51.1577342_7.2295367_ts2,"POLYGON((7.2295367, 51.1577342, 7.2296268, 51...."
4,2,12969501,None,2022-07-13T11:35:46Z,"POLYGON((10.2129962, 48.6891887, 10.2129365, 4...","@id=""relation/12969501"", amenity=""parking"", ca...",yes,surface,32.0,parking,"(48.6891887, 10.2129962)",48.6891887_10.2129962_ts2,"POLYGON((10.2129962, 48.6891887, 10.2129365, 4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,1,1231913595,None,2023-12-17T14:50:05Z,"POLYGON((8.5529195, 49.6451625, 8.5528658, 49....","@id=""way/1231913595"", access=""yes"", amenity=""p...",designated,surface,36.0,parking,"(49.6451625, 8.5529195)",49.6451625_8.5529195_ts2,"POLYGON((8.5529195, 49.6451625, 8.5528658, 49...."
944,1,1231913596,None,2023-12-17T14:50:05Z,"POLYGON((8.5534208, 49.6449368, 8.5535815, 49....","@id=""way/1231913596"", access=""yes"", amenity=""p...",designated,surface,36.0,parking,"(49.6449368, 8.5534208)",49.6449368_8.5534208_ts2,"POLYGON((8.5534208, 49.6449368, 8.5535815, 49...."
945,1,1239658430,None,2024-01-14T16:25:58Z,"POLYGON((13.5723058, 51.0616245, 13.5723061, 5...","@id=""way/1239658430"", access=""yes"", amenity=""p...",designated,street_side,NaN,parking,"(51.0616245, 13.5723058)",51.0616245_13.5723058_ts2,"POLYGON((13.5723058, 51.0616245, 13.5723061, 5..."
946,1,1240008646,None,2024-01-15T22:23:31Z,"POLYGON((8.720233, 51.6850829, 8.7202571, 51.6...","@id=""way/1240008646"", access=""private"", amenit...",yes,surface,NaN,parking,"(51.6850829, 8.720233)",51.6850829_8.720233_ts2,"POLYGON((8.720233, 51.6850829, 8.7202571, 51.6..."


In [66]:
# Loading the data and pre-processing
# This part needs to be adapted according to database used

# THIS CAN TAKE UP SOME TIME DEPENDING ON DATA SIZE

start_time = time.localtime()
start_t = time.time()
print("Started at:", time.asctime(start_time))

file_nr = [0,1] # Specify the number of files that should be merged
first = True

for i in file_nr:
    df = pd.read_json(f"osm_parking_polygons_0{i}_test.json", lines=True)
    df = df[find_tags(df)] # only keep entries that fulfil tag requirements
    df["pt"] = df["geometry"].transform(lambda x: get_point(x)) # get first point of coordinates list
    df["id"] = [parking_name(x, TS) for x in df["pt"]] # create id based on coordinates
    df['geometry'] = df['geometry'].apply(wkt.loads)
    df = df[["id", "pt", "geometry", "all_tags"]]
    df = df.dropna()

    if first:
        temp = df
        first = False
    else:
        temp = pd.concat([temp, df])

    print(f"File nr. {i} successfully loaded")

real_polys = gpd.GeoDataFrame(temp, geometry="geometry")

end_t = time.time()
print("Computation time (Min.):", (end_t-start_t)/60)

Started at: Sat Jan 27 17:12:33 2024
File nr. 0 successfully loaded
File nr. 1 successfully loaded
Computation time (Min.): 0.000665748119354248


In [67]:
real_polys

,id,pt,geometry,all_tags
1,45.4727254_3.2479538_ts2,"(45.4727254, 3.2479538)","POLYGON ((3.24795 45.47273, 3.24794 45.47277, ...","[{'key': 'access', 'value': 'yes'}, {'key': 'a..."
3,43.4387796_-112.1136612_ts2,"(43.4387796, -112.1136612)","POLYGON ((-112.11366 43.43878, -112.11375 43.4...","[{'key': 'amenity', 'value': 'parking'}, {'key..."
10,49.7030926_12.9882907_ts2,"(49.7030926, 12.9882907)","POLYGON ((12.98829 49.70309, 12.98849 49.70319...","[{'key': 'amenity', 'value': 'parking'}, {'key..."
20,38.0495663_-102.0508654_ts2,"(38.0495663, -102.0508654)","POLYGON ((-102.05087 38.04957, -102.05091 38.0...","[{'key': 'amenity', 'value': 'parking'}, {'key..."
21,35.8807974_-97.3903998_ts2,"(35.8807974, -97.3903998)","POLYGON ((-97.39040 35.88080, -97.39056 35.880...","[{'key': 'amenity', 'value': 'parking'}, {'key..."
22,51.0867363_6.8169281_ts2,"(51.0867363, 6.8169281)","POLYGON ((6.81693 51.08674, 6.81812 51.08717, ...","[{'key': 'access', 'value': 'private'}, {'key'..."
24,45.8564154_25.9405394_ts2,"(45.8564154, 25.9405394)","POLYGON ((25.94054 45.85642, 25.93991 45.85589...","[{'key': 'amenity', 'value': 'parking'}, {'key..."
0,38.0495663_-102.0508654_ts2,"(38.0495663, -102.0508654)","POLYGON ((-102.05087 38.04957, -102.05091 38.0...","[{'key': 'amenity', 'value': 'parking'}, {'key..."
1,35.4300652_-84.6883_ts2,"(35.4300652, -84.6883)","POLYGON ((-84.68830 35.43007, -84.68857 35.430...","[{'key': 'amenity', 'value': 'parking'}, {'key..."
2,38.6295002_-94.3500656_ts2,"(38.6295002, -94.3500656)","POLYGON ((-94.35007 38.62950, -94.34995 38.629...","[{'key': 'amenity', 'value': 'parking'}, {'key..."


In [ ]:
# Drop full duplicates and reset index
real_polys = real_polys.drop_duplicates(subset=["id", "pt", "geometry"])
real_polys = real_polys.reset_index(drop=True)

In [ ]:
real_polys


,id,pt,geometry,all_tags
0,45.4727254_3.2479538_ts2,"(45.4727254, 3.2479538)","POLYGON ((3.24795 45.47273, 3.24794 45.47277, ...","[{'key': 'access', 'value': 'yes'}, {'key': 'a..."
1,43.4387796_-112.1136612_ts2,"(43.4387796, -112.1136612)","POLYGON ((-112.11366 43.43878, -112.11375 43.4...","[{'key': 'amenity', 'value': 'parking'}, {'key..."
2,49.7030926_12.9882907_ts2,"(49.7030926, 12.9882907)","POLYGON ((12.98829 49.70309, 12.98849 49.70319...","[{'key': 'amenity', 'value': 'parking'}, {'key..."
3,38.0495663_-102.0508654_ts2,"(38.0495663, -102.0508654)","POLYGON ((-102.05087 38.04957, -102.05091 38.0...","[{'key': 'amenity', 'value': 'parking'}, {'key..."
4,35.8807974_-97.3903998_ts2,"(35.8807974, -97.3903998)","POLYGON ((-97.39040 35.88080, -97.39056 35.880...","[{'key': 'amenity', 'value': 'parking'}, {'key..."
5,51.0867363_6.8169281_ts2,"(51.0867363, 6.8169281)","POLYGON ((6.81693 51.08674, 6.81812 51.08717, ...","[{'key': 'access', 'value': 'private'}, {'key'..."
6,45.8564154_25.9405394_ts2,"(45.8564154, 25.9405394)","POLYGON ((25.94054 45.85642, 25.93991 45.85589...","[{'key': 'amenity', 'value': 'parking'}, {'key..."
7,35.4300652_-84.6883_ts2,"(35.4300652, -84.6883)","POLYGON ((-84.68830 35.43007, -84.68857 35.430...","[{'key': 'amenity', 'value': 'parking'}, {'key..."
8,38.6295002_-94.3500656_ts2,"(38.6295002, -94.3500656)","POLYGON ((-94.35007 38.62950, -94.34995 38.629...","[{'key': 'amenity', 'value': 'parking'}, {'key..."
9,37.9882682_58.3524541_ts2,"(37.9882682, 58.3524541)","POLYGON ((58.35245 37.98827, 58.35295 37.98898...","[{'key': 'amenity', 'value': 'parking'}, {'key..."


In [ ]:
# Create image
BOUNDS_RGB = (0xff,0x78, 0x00)

# Turn r, g, b integers into a CSS color code
def rgbcolor(r,g,b):
    return f"#{r:02x}{g:02x}{b:02x}"

# Get the bounding box around an arbitrarily shaped geometry object
def geometry_bbox(geometry, init_bounds=None, x_offset=0., y_offset=0.):
    bounds = init_bounds or [[1000,-1000],[-1000,1000]]
    xs = [c[0] for c in geometry.exterior.coords]

    ys = [c[1] for c in geometry.exterior.coords]

    return [
        #southwest
        [min(bounds[0][0], min(ys)-y_offset), max(bounds[0][1], max(xs)+x_offset)],
        #northeast
        [max(bounds[1][0], max(ys)+y_offset), min(bounds[1][1], min(xs)-x_offset)]
    ]


def make_map(pt, ts, geometry=None, show_geometry=False, size=320):
    """
    create folium map with a given center,
    optionally fit and show a geometry and/or a larger bounding box
    """

    # Create satellite image map for the coordinate point specified
    m = folium.Map(location=pt, min_zoom=15, width=size, height=size, zoom_control=False, attribution_control=False)

    # First tile server
    if ts == 1:
        folium.TileLayer(
            tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri', name = 'Esri Satellite', overlay = False, control = True
        ).add_to(m)

    # Second tile server
    elif ts == 2:
        folium.TileLayer(
            tiles = 'https://clarity.maptiles.arcgis.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            attr = 'Esri', name = 'Esri Satellite', overlay = False, control = True
        ).add_to(m)

    # Bounds of inital point
    bounds = [[pt[0], pt[1]],[pt[0], pt[1]]] #southwest, northeast

    # Plot lines of geometry
    if geometry:
        if show_geometry:
            folium.GeoJson(data=geometry).add_to(m)
            bounds_fit = geometry_bbox(geometry, init_bounds=bounds, x_offset=0, y_offset=0)

    m.fit_bounds(bounds_fit)

    return m, bounds

In [ ]:
# Sart selection
start_nr = 0 # if you stop somewhere inbetween, enter here the last image nr. printed

blacklist = []
whitelist = []
trucklist = []

for i in range(start_nr, real_polys.shape[0]):
    row = real_polys.iloc[i]
    m, b = make_map(row.pt, TS, geometry=row.geometry, show_geometry=True, size=640) # create map with boundaries

    display(m)
    inp = input(f"Image Nr. {i} ok? Just enter. If truck, type \"t\". To exit type \"exit\"")

    if inp == "exit":
        print("Image to be checked next:", row["id"])
        break
    elif inp == "t":
        trucklist.append(row["id"]) # This feature may be useful if one is looking through car images and finds truck images
    elif inp != "":
        blacklist.append(row["id"])
    else:
        whitelist.append(row["id"])

    print("Last image checked:", row["id"])
    print("")

Image Nr. 0 ok? Just enter. If truck, type "t". To exit type "exit"t
Last image checked: 45.4727254_3.2479538_ts2



Image Nr. 1 ok? Just enter. If truck, type "t". To exit type "exit"e
Last image checked: 43.4387796_-112.1136612_ts2



Image Nr. 2 ok? Just enter. If truck, type "t". To exit type "exit"t
Last image checked: 49.7030926_12.9882907_ts2



Image Nr. 3 ok? Just enter. If truck, type "t". To exit type "exit"t
Last image checked: 38.0495663_-102.0508654_ts2



Image Nr. 4 ok? Just enter. If truck, type "t". To exit type "exit"e
Last image checked: 35.8807974_-97.3903998_ts2



...

Image Nr. 8 ok? Just enter. If truck, type "t". To exit type "exit"w
Last image checked: 38.6295002_-94.3500656_ts2



Image Nr. 9 ok? Just enter. If truck, type "t". To exit type "exit"w
Last image checked: 37.9882682_58.3524541_ts2



Image Nr. 10 ok? Just enter. If truck, type "t". To exit type "exit"t
Last image checked: 43.7569066_-89.9685034_ts2



Image Nr. 11 ok? Just enter. If truck, type "t". To exit type "exit"e
Last image checked: 39.5884212_-107.1921828_ts2



Image Nr. 12 ok? Just enter. If truck, type "t". To exit type "exit"t
Last image checked: 40.5993797_-84.1575569_ts2



In [ ]:
trucklist

['45.4727254_3.2479538_ts2',
 '49.7030926_12.9882907_ts2',
 '38.0495663_-102.0508654_ts2',
 '51.0867363_6.8169281_ts2',
 '35.4300652_-84.6883_ts2',
 '43.7569066_-89.9685034_ts2',
 '40.5993797_-84.1575569_ts2']

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Other Test

In [3]:
import folium


# Link to Esri World Imagery service plus attribution
EsriImagery = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
EsriAttribution = "Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community"
MapObject = folium.Map(location=[64.9865, -18.5867], tiles=EsriImagery, attr=EsriAttribution, zoom_start=7)


In [ ]:

WeatherStationsJson = os.path.join('Vedurstodvar_VI_2017_06_26.json')  # path to GeoJSON

MapObject = folium.Map(location=[64.9865, -18.5867], tiles=EsriImagery, attr=EsriAttribution, zoom_start=7)
folium.GeoJson(WeatherStationsJson, name="IMO Weather Stations",).add_to(MapObject)
folium.LayerControl().add_to(MapObject)  # add layer control, not really needed for this example

folium.Map.save(MapObject, "index.html")  # save map as html